In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools
import geopandas as gpd

In [8]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [9]:
# simplify_dir = 'D:/RST/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'

# gtfs_dir = 'D:/RST/notebook/GTFS/TTC_2016-10-03/'

In [10]:
ward = gpd.read_file('/Volumes/Data2/RST/notebook/GIS/wards_no-island.geojson')

ward = ward[['AREA_SHORT_CODE', 'geometry']]
ward = ward.to_crs('epsg:26917')
ward['area'] = ward['geometry'].area/1000000

ward = ward.rename(columns = {'AREA_SHORT_CODE':'ward'})

In [11]:
ward_int = pd.read_csv('/Volumes/Data2/RST/notebook/GIS/int_wards.csv')

#ward_int = pd.read_csv('D:/RST/notebook/GIS/int_wards.csv')

ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [13]:
# G = nx.read_gexf('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.gexf')
# graph_data = pd.read_csv('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.csv')

stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [56]:
df_list = []

period_list = [['EM', [3,4]], ['AM', [7,8]], ['MD', [11,12]], ['PM', [16,17]], ['EV', [20,21]]]

for i in period_list:

    period = i[0]
    period_list = i[1]


    G = nx.read_gexf('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16.gexf')
    graph_data = pd.read_csv('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16.csv')

    route_dir = stop_times_od[stop_times_od['hr_o'].isin(period_list)]
    route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()

    route_dir = route_dir.append(route_dir.rename(columns = {'INT_ID_o':'INT_ID_d', 'INT_ID_d':'INT_ID_o'})[['INT_ID_o', 
                                                                                'INT_ID_d', 'route_short_name']])

    route_dir = route_dir.drop_duplicates()


    route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
    route_dir = route_dir.merge(graph_data)

    for ward in range(1,26,1):

        ints_in_ward = list(ward_int[ward_int['AREA_SHORT_CODE'] == ward]['INT_ID'])
        not_in_ward = list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR'])

        G_ward = G.copy()

        G_ward.remove_nodes_from(not_in_ward)

        route_dir_ward = route_dir[route_dir['INT_ID_o'].isin(ints_in_ward)]
        route_dir_ward = route_dir_ward[route_dir_ward['INT_ID_d'].isin(ints_in_ward)]

        EF = route_dir_ward['scaled_freq'].sum()
        ES = len(list(G_ward.edges()))
        V = len(list(G_ward.nodes()))

        gamma = EF * (ES/(3 * (V-2)))

        df_list.append([ward, EF, ES, V, gamma, period])

In [58]:
pd.DataFrame.from_records(df_list, columns = ['ward', 'ef', 'es', 'v', 'gamma', 'period']).to_csv(
    'degree-of-connectivity.csv', index = False)